In [2]:
import pandas as pd
tract_join=pd.read_csv('../Datasets/ALL_watersystems_tracts_intersection.txt',encoding = "ISO-8859-1")
tract_info=tract_join[['PWSID','D_POPULATI','GEOID10','ALAND10','AWATER10','AREA_TRACT','AREA_Intersection','Proportion_Int_Tract']]
caldata=pd.read_excel('../Datasets/calenviroscreen.xlsx','CES 3.0 (2018 Update)')

In [3]:
#Read in the CWS data
cws=pd.read_csv('../Datasets/CWS.csv').dropna(subset=['Water System No.'])
def mk_pwsid(row):
    return('CA{:07d}'.format(int(row['Water System No.'])))
cws.loc[:,'PWSID']=cws.apply(mk_pwsid,axis=1)
cws_tract=pd.merge(cws,tract_info)

In [4]:
calsub=caldata[['Census Tract','Total Population','CES 3.0 Score']]
cal_tract=pd.merge(calsub,cws_tract,left_on='Census Tract',right_on='GEOID10')
cal_tract.loc[:,'Tract_Frac_Pop']=cal_tract['Proportion_Int_Tract']*cal_tract['Total Population']

In [6]:
cal_merge=cal_tract[['PWSID','Water System Name','Regulating Agency','Fee Code','Service Connections',
                     'Population','Census Tract','Total Population','Proportion_Int_Tract','Tract_Frac_Pop','CES 3.0 Score']]

In [43]:
#cal_merge.to_excel('../Datasets/System_Tract_Merge_Info.xlsx',index=False)

In [45]:
#Read in the ACS median income
acs_hh_median=pd.read_csv('../Datasets/ACS/ACS_Median_HH.csv')
acs_hh_median.loc[:,'Census Tract']=pd.to_numeric(acs_hh_median['GEO_ID'].str.slice(10,21))
acs_hh_rent=pd.read_csv('../Datasets/ACS/ACS_Rent_Pct_Income.csv')
acs_hh_rent.loc[:,'Census Tract']=pd.to_numeric(acs_hh_rent['GEO_ID'].str.slice(10,21))

In [47]:
acs_1=pd.merge(cal_merge,acs_hh_median[['Census Tract','Median_12month_HH_Income']],on='Census Tract')
acs_2=pd.merge(acs_1,acs_hh_rent[['Census Tract','Median_rent_pct_Income']])

In [49]:
acs_cal=acs_2

In [50]:
#Population weighted values?
sys_stats=pd.DataFrame()
field_calcs=['CES 3.0 Score','Median_12month_HH_Income','Median_rent_pct_Income']
for p in acs_cal['PWSID'].unique():
    csub=acs_cal[acs_cal['PWSID']==p]
    cname=csub['Water System Name'].values[0]
    sys_stats.loc[p,'NAME']=cname
    sys_stats.loc[p,'Fee Code']=csub['Fee Code'].values[0]
    #sys_stats.loc[p,'N']=len(csub)
    sys_stats.loc[p,'Population']=csub['Population'].mean()
    sys_stats.loc[p,'Service Connections']=csub['Service Connections'].mean()
    if (len(csub)<2):

        sys_stats.loc[p,'Population']=csub['Population'].values[0]
        for c in field_calcs:
            sys_stats.loc[p,c]=csub[c].values[0]
    else:

        pop_tracts=csub['Tract_Frac_Pop'].sum()
        for c in field_calcs:
            #print(csub[['Tract_Frac_Pop',c]])
            #print((csub['Tract_Frac_Pop']*csub[c]).sum()/csub['Tract_Frac_Pop'].sum())
            sys_stats.loc[p,c]=(csub['Tract_Frac_Pop']*csub[c]).sum()/csub['Tract_Frac_Pop'].sum()
        

In [52]:
#sys_stats
sys_stats.to_excel('../Datasets/pop_weighted_characteristics.xlsx')